In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("merchant_fill_nonulls.csv", index_col = 0)
df.head()

,title,price,retail_price,units_sold,uses_ad_boosts,rating_sold,rating_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_id,product_url,product_id,listed_products,total_units_sold,mean_units_sold_per_product,rating_employee,merchant_ratings_count,mean_product_prices,mean_retail_prices,mean_product_ratings_count,discount_percentage,seller_reputation,rating_four_count,rating_three_count,rating_two_count,rating_one_count,rating_five_count
0,2020 Summer Vintage Flamingo Print Pajamas Se...,16.00,14,100,No,3.76,54,No,Buena,No,"Summer,Fashion,womenunderwearsuit,printedpajam...",white,M,Livraison standard,4,No,34,50,CN,zgrdejia,Unknown,568,595097d6a26f6e070cb878d1,https://www.wish.com/c/5e9ae51d43d6a96e303acdb0,5e9ae51d43d6a96e303acdb0,1,100,100.0,4.129,568.0,16.00,14.00,54.0,14.29,Buena,8.0,10.0,1.0,9.0,26.0
1,SSHOUSE Summer Casual Sleeveless Soirée Party ...,8.00,22,20000,Si,3.45,6135,No,Buena,No,"Mini,womens dresses,Summer,Patchwork,fashion d...",green,XS,Livraison standard,2,No,41,50,CN,sarahouse,83 % avis positifs,17752,56458aa03a698c35c9050988,https://www.wish.com/c/58940d436a0d3d5da4e95a38,58940d436a0d3d5da4e95a38,6,21400,3567.0,3.900,17752.0,5.68,10.33,1057.0,-63.64,Regular,1027.0,1118.0,644.0,1077.0,2269.0
2,Nouvelle mode d'été femmes robe décontractée c...,4.90,8,1000,Si,3.83,99,No,Buena,No,"Summer,Fashion,Necks,Beach,Dress,Loose,beach d...",white,XXS,Livraison standard,1,No,41,50,CN,sarahouse,83 % avis positifs,17752,56458aa03a698c35c9050988,https://www.wish.com/c/5df2576d68963c1660471f9a,5df2576d68963c1660471f9a,6,21400,3567.0,3.900,17752.0,5.68,10.33,1057.0,-38.75,Regular,18.0,23.0,8.0,7.0,43.0
3,Summer Women s Fashion Lace Up Tie Pants Plus...,4.93,6,100,No,3.77,47,No,Buena,No,"Summer,Shorts,Lace,pants,Waist,Short pants,Yel...",red,S,Standard Shipping,1,No,41,50,CN,sarahouse,83% Positive Feedback,17752,56458aa03a698c35c9050988,https://www.wish.com/c/5d58daef3159a812b05933d2,5d58daef3159a812b05933d2,6,21400,3567.0,3.900,17752.0,5.68,10.33,1057.0,-17.83,Regular,7.0,8.0,5.0,5.0,22.0
4,Nouvelle mode d'été femmes robe décontractée c...,5.65,10,100,Si,2.86,7,No,Buena,No,"Summer,Fashion,Necks,Skirts,Dress,Loose,Women'...",black,XXS,Livraison standard,1,No,41,50,CN,sarahouse,83 % avis positifs,17752,56458aa03a698c35c9050988,https://www.wish.com/c/5e9fa7de39682a0043ab7898,5e9fa7de39682a0043ab7898,6,21400,3567.0,3.900,17752.0,5.68,10.33,1057.0,-43.50,Regular,1.0,2.0,2.0,1.0,1.0


# Ejercicios

Seguiremos usando el conjunto de datos que usamos siempre para los ejercicios de la clase invertida. 

1. Ejercicios intervalo de confianza: Para cada uno de los ejercicios interpreta los resultados y piensa que ventaja a nivel de negocio podemos sacar de calcularlo. 

   - Trabajas para una plataforma de comercio en línea y deseas determinar el intervalo de confianza al 95% para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500.

   - Eres un analista de ventas en una plataforma de comercio electrónico y quieres calcular el intervalo de confianza al 90% para el número promedio de unidades vendidas por producto.

   - Trabajas en la gestión de inventario de una tienda en línea y necesitas calcular el intervalo de confianza al 99% para el precio promedio de productos según su país de origen.

   - Eres un gerente de ventas en una plataforma de comercio electrónico y quieres determinar el intervalo de confianza al 95% para la valoración promedio de los vendedores según si ofrecen envío express o no.

   - Trabajas en el departamento de precios de una tienda en línea y necesitas calcular el intervalo de confianza al 90% para el descuento promedio de productos según la reputación del vendedor (alta, media o baja).

In [ ]:
def intervalo_confianza(dataframe, columna, nivel_confianza = 0.95): # si no se especifica dejamos un nivel confianza 95% por defecto
    
    """
    Calcula el intervalo de confianza para una columna específica en un DataFrame.

    Parámetros:
    -----------
    dataframe : DataFrame
        El DataFrame que contiene los datos de la muestra.
    columna : str
        El nombre de la columna para la cual deseas calcular el intervalo de confianza.
    nivel_confianza : float, opcional
        El nivel de confianza deseado para el intervalo de confianza (valor predeterminado es 0.95).

    Salida:
    -------
    None
        La función imprime en la consola la siguiente información:
        - La media muestral de la columna especificada.
        - La desviacion tipica muestral de la columna especificada
        - El nivel de confianza utilizado.
        - El intervalo de confianza calculado, que incluye el límite inferior y el límite superior.
    """

    # Calcular la media y el error estándar de la muestra
    media = dataframe[columna].mean()
    desviacion_estandar = dataframe[columna].std()

    # calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
    tamaño_muestra = len(df[columna]) 
    grados_libertad = tamaño_muestra - 1
    
    
    # Calcular el intervalo de confianza al 95%
    confidence_interval = stats.t.interval(
        confidence=nivel_confianza,         # Nivel de confianza
        df = grados_libertad,             # Grados de libertad. Es para corregir el sesgo. Siempre lo ponemos así
        loc= media,  # Media de la muestra
        scale = desviacion_estandar/ np.sqrt(tamaño_muestra))  # Adaptamos desviación estándar a la muestra


    print(f"Intervalo de Confianza para {columna}:")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Intervalo de Confianza: {confidence_interval}")
    

In [4]:
# Otra formula diferente para intervalos de confianza (stats.t.ppf). 
# La diferencia de stats.t.ppf y stats.t.interval es que el primero requiere calcular el error estandar, 
# el valor critico, y los limites del intervalo de confianza superior e inferior (además de la media,
# desviacion estandar y grados de libertad). En cambio stats.t.interval nos da directamente el intervalo y requiere menos calculos

def intervalo_confianza_2(dataframe, columna, nivel_confianza = float):
    
    """
    Calcula el intervalo de confianza para una columna específica en un DataFrame.

    Parámetros:
    -----------
    dataframe : DataFrame
        El DataFrame que contiene los datos de la muestra.
    columna : str
        El nombre de la columna para la cual deseas calcular el intervalo de confianza.
    nivel_confianza : float, opcional
        El nivel de confianza deseado para el intervalo de confianza (valor predeterminado es 0.95).

    Salida:
    -------
    None
        La función imprime en la consola la siguiente información:
        - La media muestral de la columna especificada.
        - El error estándar de la muestra.
        - El nivel de confianza utilizado.
        - El valor crítico calculado a partir de la distribución t de Student.
        - El intervalo de confianza calculado, que incluye el límite inferior y el límite superior.
    """

    # Calcular la media y el error estándar de la muestra
    media = dataframe[columna].mean()
    error = stats.sem(dataframe[columna])

    # calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
    grados_libertad = len(df[columna]) - 1

    # Calcular el valor crítico (utilizando la distribución t de Student)
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)

    # Calcular el intervalo de confianza
    limite_inferior = media - valor_critico * error
    limite_superior = media + valor_critico * error

    print(f"Intervalo de Confianza para {columna}:")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Error Estándar: {np.round(error, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Valor Crítico: {np.round(valor_critico, 2)}")
    print(f"Intervalo de Confianza: ({np.round(limite_inferior, 2)}, {np.round(limite_superior, 2)})")
    

In [5]:
# 2: Trabajas para una plataforma de comercio en línea y deseas determinar el intervalo de confianza al 95% para el precio promedio de 
# productos con una cantidad de media de valoraciones al producto de 500.

# Creamos mascara filtrando el dataframe para que tenga solo los datos que nos interesan, con mean_product_ratings_count > 500
df_filtrado = df[df["mean_product_ratings_count"] > 500]
df_filtrado.head(1)

,title,price,retail_price,units_sold,uses_ad_boosts,rating_sold,rating_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_id,product_url,product_id,listed_products,total_units_sold,mean_units_sold_per_product,rating_employee,merchant_ratings_count,mean_product_prices,mean_retail_prices,mean_product_ratings_count,discount_percentage,seller_reputation,rating_four_count,rating_three_count,rating_two_count,rating_one_count,rating_five_count
1,SSHOUSE Summer Casual Sleeveless Soirée Party ...,8.0,22,20000,Si,3.45,6135,No,Buena,No,"Mini,womens dresses,Summer,Patchwork,fashion d...",green,XS,Livraison standard,2,No,41,50,CN,sarahouse,83 % avis positifs,17752,56458aa03a698c35c9050988,https://www.wish.com/c/58940d436a0d3d5da4e95a38,58940d436a0d3d5da4e95a38,6,21400,3567.0,3.9,17752.0,5.68,10.33,1057.0,-63.64,Regular,1027.0,1118.0,644.0,1077.0,2269.0


In [6]:
intervalo_confianza(df, "mean_product_prices", 0.95)

Intervalo de Confianza para mean_product_prices:
Media Muestral: 8.33
Nivel de Confianza: 0.95
Intervalo de Confianza: (8.148130150552152, 8.503071375194825)


Esto puede ayudar a tomar decisiones de fijación de precios y estrategias de marketing específicas para productos de alta calificación.

In [7]:
# Eres un analista de ventas en una plataforma de comercio electrónico y quieres calcular el intervalo de confianza al 90% para el número promedio de unidades vendidas por producto.
intervalo_confianza(df, "mean_units_sold_per_product", 0.90 )

Intervalo de Confianza para mean_units_sold_per_product:
Media Muestral: 4339.01
Nivel de Confianza: 0.9
Intervalo de Confianza: (4005.0527419490304, 4672.96760134404)


Esto es esencial para la gestión de inventario, la previsión de la demanda y la optimización de la oferta de productos.

In [8]:
# Trabajas en la gestión de inventario de una tienda en línea y necesitas calcular el intervalo de confianza al 99% para el precio promedio de productos según su país de origen.

# lo primero que tenemos que hacer es crear un dataframe para cada pais de origen, y luego para cada uno de ellos calcular el intervalo de confianza

diccionario_paises = {}

for pais in df["origin_country"].unique():
    diccionario_paises[pais] = df[df["origin_country"] == pais]

for k, v in diccionario_paises.items():
    if v.shape[0] > 1:
        print(f"Analisis para el pais {k}")
        print(intervalo_confianza(v, "mean_product_prices", 0.99 ))
        print("-------------")
    else:
        print(f"Para {k} no se puede calcular el intervalo de confianza por que no hay datos suficientes")


Analisis para el pais CN
Intervalo de Confianza para mean_product_prices:
Media Muestral: 8.31
Nivel de Confianza: 0.99
Intervalo de Confianza: (8.076996501093673, 8.544803890295762)
None
-------------
Analisis para el pais US
Intervalo de Confianza para mean_product_prices:
Media Muestral: 9.4
Nivel de Confianza: 0.99
Intervalo de Confianza: (9.217116236036555, 9.577077312350541)
None
-------------
Analisis para el pais VE
Intervalo de Confianza para mean_product_prices:
Media Muestral: 8.3
Nivel de Confianza: 0.99
Intervalo de Confianza: (7.968510776813829, 8.62748922318617)
None
-------------
Para AT no se puede calcular el intervalo de confianza por que no hay datos suficientes
Analisis para el pais SG
Intervalo de Confianza para mean_product_prices:
Media Muestral: 3.38
Nivel de Confianza: 0.99
Intervalo de Confianza: (nan, nan)
None
-------------
Para GB no se puede calcular el intervalo de confianza por que no hay datos suficientes



NOTA: hay algunos paises que sale nan porque solo hay dato. 

Esto puede guiar la estrategia de precios, la selección de proveedores y la comunicación con los clientes sobre la procedencia de los productos.

In [9]:
# Eres un gerente de ventas en una plataforma de comercio electrónico y quieres determinar el intervalo de confianza al 95% para la valoración promedio de los vendedores según si ofrecen envío express o no.
diccionario_tipo_envio = {}

for envio in df["shipping_is_express"].unique():
    diccionario_tipo_envio[envio] = df[df["shipping_is_express"] == envio]

for k, v in diccionario_tipo_envio.items():

    print(f"Analisis para el tipo de envio {k}")
    print(intervalo_confianza(v, "rating_employee", 0.95 ))
    print("-------------")
  

Analisis para el tipo de envio No
Intervalo de Confianza para rating_employee:
Media Muestral: 4.03
Nivel de Confianza: 0.95
Intervalo de Confianza: (4.021608617778935, 4.041847086491301)
None
-------------
Analisis para el tipo de envio Si
Intervalo de Confianza para rating_employee:
Media Muestral: 4.28
Nivel de Confianza: 0.95
Intervalo de Confianza: (4.279396469781684, 4.284603530218316)
None
-------------


Ayuda a identificar si los vendedores que ofrecen envío express tienen calificaciones significativamente mejores, lo que puede influir en la elección de proveedores y políticas de envío.

In [10]:
# Trabajas en el departamento de precios de una tienda en línea y necesitas calcular el intervalo de confianza al 90% para el descuento promedio de productos según la reputación del vendedor (alta, media o baja).

# lo primero que hay que hacer es categorizar la columna de "rating_employee" en alta media y baja

def categorizar_rating_emplo(col):
    if col < 2:
        return "baja"
    elif col >= 2 and col <= 4:
        return "media"
    else:
        return "alta"
df["employee_rating"] = df["rating_employee"].apply(categorizar_rating_emplo)
        

In [11]:
diccionario_rating_emplo = {}

for rating in df["employee_rating"].unique():
    diccionario_rating_emplo[rating] = df[df["employee_rating"] == rating]

for k, v in diccionario_rating_emplo.items():

    print(f"Analisis para el rating del empleado {k}")
    print(intervalo_confianza(v, "discount_percentage", 0.90 ))
    print("-------------")

Analisis para el rating del empleado alta
Intervalo de Confianza para discount_percentage:
Media Muestral: -26.56
Nivel de Confianza: 0.9
Intervalo de Confianza: (-28.25909388101557, -24.86045255526952)
None
-------------
Analisis para el rating del empleado media
Intervalo de Confianza para discount_percentage:
Media Muestral: -25.35
Nivel de Confianza: 0.9
Intervalo de Confianza: (-27.03982218708757, -23.652851692356013)
None
-------------


Esto puede guiar las estrategias de negociación con los vendedores y la promoción de productos en función de la reputación del vendedor.

3. Ejercicios prueba de hipótesis:

   - Prueba de Hipótesis sobre el precio promedio de productos con o sin empleados calificados

      - *Contexto:* Trabajas en una plataforma de comercio en línea que ofrece productos de moda. Quieres saber si hay una diferencia significativa en el precio promedio de productos entre aquellos vendidos por empleados calificados (rating_employee > 4) y aquellos vendidos por empleados menos calificados (rating_employee <= 4). Esto te ayudará a tomar decisiones sobre la estrategia de precios y capacitación de empleados.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el precio promedio de productos entre empleados con buena y mala reputación. 
      
      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el precio promedio de productos entre empleados calificados y menos calificados.

   - Prueba de Hipótesis sobre el número de unidades vendidas en envío express y envío estándar

      - *Contexto:* Eres un gerente de operaciones de envío en una plataforma de comercio electrónico. Quieres determinar si el número promedio de unidades vendidas difiere cuando los productos se envían con envío express (shipping_option_name = "Express Shipping") en comparación con envío estándar (shipping_option_name = "Livraison standard"). Esto te ayudará a tomar decisiones sobre la logística de envío.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.

      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.

In [12]:
# vamos a definir una función para poder hacer el t-test de Student
## vamos a definir una superfunción con todo el proceso

def prueba_hipotesis(*args):
    """
    Realiza una prueba de hipótesis para comparar grupos.
    1. Primero verifica si los datos son normales usando el test de Shapiro-Wilk o Kolmogorov-Smirnov.
    2. Si los datos son normales, usa Bartlett para probar igualdad de varianzas. Si no son normales, usa Levene.
    3. Si las varianzas son iguales, usa el t-test de Student; si no, usa la versión de Welch.
    4. Si los datos no son normales, usa el test de Mann-Whitney

    Parámetros:
    *args: listas o arrays con los datos de cada grupo. Espera DOS grupos a comparar

    Retorna:
    dict con resultados del test de normalidad, varianza e hipótesis.
    """
    
    # Verificar si hay al menos dos grupos
    if len(args) < 2:
        raise ValueError("Se necesitan al menos dos conjuntos de datos para realizar la prueba.")
    
    # Comprobar normalidad en cada grupo
    normalidad = []
    for grupo in args:
        if len(grupo) > 50:
            p_valor_norm = stats.kstest(grupo, 'norm').pvalue  # Kolmogorov-Smirnov si n > 50
        else:
            p_valor_norm = stats.shapiro(grupo).pvalue  # Shapiro-Wilk si n <= 50
        normalidad.append(p_valor_norm > 0.05)

    datos_normales = all(normalidad)  # True si todos los grupos son normales

    # Prueba de igualdad de varianzas
    if datos_normales:
        p_valor_varianza = stats.bartlett(*args).pvalue  # Test de Bartlett si los datos son normales
    else:
        p_valor_varianza = stats.levene(*args, center="median").pvalue  # Test de Levene si no son normales

    varianzas_iguales = p_valor_varianza > 0.05

    # Aplicar el test adecuado
    if datos_normales:
        if varianzas_iguales:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
            test_usado = "t-test de Student (varianzas iguales)"
        else:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
            test_usado = "t-test de Welch (varianzas desiguales)"
    else:
        t_stat, p_valor = stats.mannwhitneyu(*args)
        test_usado = "Mann-Whitney U"

    # Nivel de significancia
    alfa = 0.05

    # Resultados
    resultado = {
        "Test de Normalidad": normalidad,
        "Datos Normales": datos_normales,
        "p-valor Varianza": p_valor_varianza,
        "Varianzas Iguales": varianzas_iguales,
        "Test Usado": test_usado,
        "Estadístico": t_stat,
        "p-valor": p_valor,
        "Conclusión": "Rechazamos H0. Es decir, sí hay diferencias significativas)" if p_valor < alfa else "No se rechaza H0. Es decir, no hay diferencias significativas)"
    }

    # Imprimir resultados de manera más clara
    print("\n📊 **Resultados de la Prueba de Hipótesis** 📊")
    print(f"✅ Test de Normalidad: {'Sí' if datos_normales else 'No'}")
    print(f"   - Normalidad por grupo: {normalidad}")
    print(f"✅ Test de Varianza: {'Iguales' if varianzas_iguales else 'Desiguales'} (p = {p_valor_varianza:.4f})")
    print(f"✅ Test aplicado: {test_usado}")
    print(f"📉 Estadístico: {t_stat:.4f}, p-valor: {p_valor:.4f}")
    print(f"🔍 Conclusión: {resultado['Conclusión']}\n")



In [13]:
# Prueba de Hipótesis sobre el precio promedio de productos con o sin empleados calificados
df["employee_rat_cat"] = np.where(df["rating_employee"] > 4, "calificado", "no-calificado")
df_calificado = df[df["employee_rat_cat"] == "calificado"]["mean_product_prices"]
df_no_calificado = df[df["employee_rat_cat"] == "no-calificado"]["mean_product_prices"]

In [14]:
prueba_hipotesis(df_calificado, df_no_calificado)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = 0.0106)
✅ Test aplicado: Mann-Whitney U
📉 Estadístico: 310055.0000, p-valor: 0.2358
🔍 Conclusión: No se rechaza H0. Es decir, no hay diferencias significativas)



In [15]:
df.columns

Index(['title', 'price', 'retail_price', 'units_sold', 'uses_ad_boosts',
       'rating_sold', 'rating_count', 'badge_local_product',
       'badge_product_quality', 'badge_fast_shipping', 'tags', 'product_color',
       'product_variation_size_id', 'shipping_option_name',
       'shipping_option_price', 'shipping_is_express', 'countries_shipped_to',
       'inventory_total', 'origin_country', 'merchant_name',
       'merchant_info_subtitle', 'merchant_rating_count', 'merchant_id',
       'product_url', 'product_id', 'listed_products', 'total_units_sold',
       'mean_units_sold_per_product', 'rating_employee',
       'merchant_ratings_count', 'mean_product_prices', 'mean_retail_prices',
       'mean_product_ratings_count', 'discount_percentage',
       'seller_reputation', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count', 'rating_five_count',
       'employee_rating', 'employee_rat_cat'],
      dtype='object')

In [16]:
df.columns

Index(['title', 'price', 'retail_price', 'units_sold', 'uses_ad_boosts',
       'rating_sold', 'rating_count', 'badge_local_product',
       'badge_product_quality', 'badge_fast_shipping', 'tags', 'product_color',
       'product_variation_size_id', 'shipping_option_name',
       'shipping_option_price', 'shipping_is_express', 'countries_shipped_to',
       'inventory_total', 'origin_country', 'merchant_name',
       'merchant_info_subtitle', 'merchant_rating_count', 'merchant_id',
       'product_url', 'product_id', 'listed_products', 'total_units_sold',
       'mean_units_sold_per_product', 'rating_employee',
       'merchant_ratings_count', 'mean_product_prices', 'mean_retail_prices',
       'mean_product_ratings_count', 'discount_percentage',
       'seller_reputation', 'rating_four_count', 'rating_three_count',
       'rating_two_count', 'rating_one_count', 'rating_five_count',
       'employee_rating', 'employee_rat_cat'],
      dtype='object')

In [17]:
# Prueba de Hipótesis sobre el número de unidades vendidas en envío express y envío estándar

df_express = df[df['shipping_is_express'] == 'Yes']["units_sold"]
df_no_express = df_express = df[df['shipping_is_express'] == 'No']["units_sold"]

prueba_hipotesis(df_express, df_no_express)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Iguales (p = 1.0000)
✅ Test aplicado: Mann-Whitney U
📉 Estadístico: 1230880.5000, p-valor: 1.0000
🔍 Conclusión: No se rechaza H0. Es decir, no hay diferencias significativas)

